In [1]:
import numpy as np
from dataset import SP_N as sp
import cvxpy as cp
import matplotlib.pyplot as plt
from portfolio import Portfolio as pf
import estimation as est

ndays = 1350
d = sp(ndays, "2012-06-01")

(CVXPY) Jun 04 10:01:34 PM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


In [7]:
n = 100
na = 5
nb = 10

a = np.zeros((n, ndays, 5))
for k in range(n):
    a[k], dates = d[k]
print(a.shape)

def split(data):
    splitv = .7
    sint = int(splitv * ndays)
    train_d = data[:, :sint]
    test_d = data[:, sint - 10:]
    return train_d, test_d

train_d, test_d = split(a)
print(train_d.shape)
print(test_d.shape)

(100, 1350, 5)
(100, 944, 5)
(100, 416, 5)


In [12]:
thta = []
output = []
rho = .98
c = .4

mout = np.zeros((n, na, test_d.shape[1] - na - nb + 1))
for k in range(n):
    ph1, ph2, ph3 = est.makearmodel(train_d[k], na, nb, rho, c, "rls", False)
    thta.append(ph1)
    mout[k], ph2 = est.priceest(test_d[k], thta[k], na, nb)
    output.append(ph2)

In [13]:
din, dout = est.getdata(train_d[0], 5, 10)
print(din.shape)
print(dout.shape)

print(mout.shape)
print(test_d.shape)

def returns(data):
    ret = np.zeros((data.shape[0], data.shape[1] - 1))
    for k in range(data.shape[0]):
        ret[k] = data[k, 1:] / data[k, :-1]
    return ret
cov = np.cov(returns(train_d[:, -41:, 3]))
print(cov.shape)

(930, 40)
(930, 5)
(100, 5, 402)
(100, 416, 5)
(100, 100)


In [22]:
initv = 2000
days = 240

In [23]:
# Single Int
pi = pf(initv, days + 10, 1, n, cov, .00001, train_d[:, -1, 3], 300, False)
returnsi = np.zeros((days,))
dvali = np.zeros((days,))
for k in range(days):
    pi.trade(mout[:, 0, k], False, True)
    dvali[k], returnsi[k] = pi.analyze(test_d[:, k, 3])

Day 0 - Optimization
Gamma - 300


Day 1 - Analysis
Estimate Value at t=1 - 2009.5457008949202
Value at  - 2001.0584816464998
Daily Return - 0.05292408232499213%
Total Return - 0.05292408232499213%
Max Daily Return - 2.947255748315468%
Gross Exposure - 555.0400161743164
Leverage at - 0.27737321086069483
Turnover at - 0.13849899864196777


Day 1 - Optimization
Gamma - 300


Day 2 - Analysis
Estimate Value at t=2 - 2195.4356795807935
Value at  - 2029.701935639543
Daily Return - 1.4314151363270042%
Total Return - 1.4850967819771625%
Max Daily Return - 6.699485856308156%
Gross Exposure - 1589.5850191116333
Leverage at - 0.7831617988829318
Turnover at - 0.5287166771550316


Day 2 - Optimization
Gamma - 300


Day 3 - Analysis
Estimate Value at t=3 - 2212.6243146581696
Value at  - 2041.3197537810977
Daily Return - 0.5723903563157506%
Total Return - 2.065987689054893%
Max Daily Return - 2.0968457701801144%
Gross Exposure - 1552.9119691848755
Leverage at - 0.76073920624559
Turnover at - 0.25696

In [21]:
# Single
p = pf(initv, days + 10, 1, n, cov, .00001, train_d[:, -1, 3], 300, False)
returns = np.zeros((days,))
dval = np.zeros((days,))
for k in range(days):
    p.trade(mout[:, 0, k], False, False)
    dval[k], returns[k] = p.analyze(test_d[:, k, 3])

Day 0 - Optimization
Gamma - 300


Day 1 - Analysis
Estimate Value at t=1 - 3016.558501537247
Value at  - 2999.94639430794
Daily Return - -0.001786856402006265%
Total Return - -0.001786856402006265%
Max Daily Return - 2.947255748315468%
Gross Exposure - 1025.087191173738
Leverage at - 0.3417018361123803
Turnover at - 0.1708600908767241


Day 1 - Optimization
Gamma - 300


Day 2 - Analysis
Estimate Value at t=2 - 3379.372894817993
Value at  - 3062.3095594111364
Daily Return - 2.078809315443891%
Total Return - 2.0769853137045544%
Max Daily Return - 6.699485856308156%
Gross Exposure - 3062.309558937872
Leverage at - 0.9999999998454552
Turnover at - 0.6453836234704622


Day 2 - Optimization
Gamma - 300


Day 3 - Analysis
Estimate Value at t=3 - 3392.3830530006844
Value at  - 3077.3439083092117
Daily Return - 0.49094804448725426%
Total Return - 2.5781302769737238%
Max Daily Return - 2.0968457701801144%
Gross Exposure - 2951.141504212573
Leverage at - 0.9589898276380887
Turnover at - 0.32138

In [ ]:
# Single Oracle
po = pf(initv, days + 10, 1, n, cov, .00001, train_d[:, -1, 3], 300, False)
returns_o = np.zeros((days,))
dval_o = np.zeros((days,))
for k in range(days):
    po.trade(test_d[:, k, 3], False, True)
    dval_o[k], returns_o[k] = po.analyze(test_d[:, k, 3])

In [ ]:
# Mulit Int
pm = pf(initv, days + 10, 5, n, cov, .00001, train_d[:, -1, 3], .02, False)
returns_m = np.zeros((days,))
dval_m = np.zeros((days,))
for k in range(days):
    pm.trade(mout[:, :, k].T, False, True, .2)
    dval_m[k], returns_m[k] = pm.analyze(test_d[:, k, 3])